In [63]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import Dataset, load_dataset, Audio
import soundfile as sf
import torch
from jiwer import cer
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

In [58]:
ds = Dataset.from_dict(
    {
        "audio": [
            "/Users/jongbeom.kim/project/corpus_raw/b2b_projects/2022/2022-PB-01/male/delivery/All_Light_Off/unknown_male_(20)_c_026_normal_000126.wav",
            "/Users/jongbeom.kim/project/corpus_raw/b2b_projects/2022/2022-PB-01/male/delivery/All_Light_Off/unknown_male_(20)_c_026_normal_000226.wav",
            "/Users/jongbeom.kim/project/corpus_raw/b2b_projects/2022/2022-PB-01/male/delivery/All_Light_Off/unknown_male_(20)_c_026_normal_000326.wav",
            "/Users/jongbeom.kim/project/corpus_raw/b2b_projects/2022/2022-PB-01/male/delivery/All_Light_Off/unknown_male_(20)_c_026_normal_000426.wav"
        ]
    }
).cast_column("audio", Audio())

In [59]:
def add_text(batch):
    batch["file"] = batch["audio"]["path"]
    audio_path = batch["audio"]["path"]
    txt_path = Path(str(audio_path).replace("wav", "txt"))
    # print(txt_path)
    # print(open(txt_path, mode="r").read())
    batch["text"] = open(txt_path, mode="r").read()
    return batch


def map_to_array(batch):
    # speech, _ = sf.read(batch["file"])
    # print(batch["audio"]["p)
    speech, _ = sf.read(batch["file"])
    # print(speech)
    batch["speech"] = speech
    return batch


def map_to_pred(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    # input_values = inputs.input_values.to("cuda")
    input_values = inputs.input_values
    
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

In [9]:

# processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

# # model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')
# model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
# ds = load_dataset(path="kresnik/zeroth_korean", name="clean", split="test")


processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [64]:

test_ds = ds
test_ds = test_ds.map(add_text)
test_ds = test_ds.map(map_to_array)

result = test_ds.map(map_to_pred, batched=True, batch_size=16, remove_columns=["speech"])

print(cer(result["text"], result["transcription"]))

100%|██████████| 1/1 [00:03<00:00,  3.61s/ba]

0.9310344827586207


In [65]:
result["text"]

['일괄 소등', '일괄 소등 (약간빠른속도)', '일괄 소등', '일괄 소등']

In [66]:
result["transcription"]

['아  수 다', '습니', '이 습대', '일끈다']